#### Friday, November 1, 2024

[Build Vector Stores and Retrievers](https://python.langchain.com/docs/tutorials/retrievers/)

mamba activate langchain-chroma

Since we want to use LMStudio, we will not be using OpenAI Embeddings, but will call some embedding model through LMStudio.

Running the below curl command from the command line works ... 
    
    curl http://127.0.0.1:1234/v1/embeddings \
    -H  "Content-Type: application/json" \
    -d '{
        "model": "text-embedding-nomic-embed-text-v1.5",
        "input": "Embed this string for me!"
    }'


In [ ]:
import os

print(os.environ["LANGCHAIN_TRACING_V2"])
print(os.environ["LANGCHAIN_API_KEY"])
print(os.environ["OPENAI_API_KEY"])

#### Documents

LangChain implements a Document abstraction, which is intended to represent a unit of text and associated metadata. It has two attributes:

- page_content: a string representing the content
- metadata: a dict containing arbitrary metadata

The metadata attribute can capture information about the source of the document, its relationship to other documents, and other information. Note that an individual Document object often represents a chunk of a larger document.

Let's generate some sample documents:

In [2]:
from langchain_core.documents import Document

documents = [
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Goldfish are popular pets for beginners, requiring relatively simple care.",
        metadata={"source": "fish-pets-doc"},
    ),
    Document(
        page_content="Parrots are intelligent birds capable of mimicking human speech.",
        metadata={"source": "bird-pets-doc"},
    ),
    Document(
        page_content="Rabbits are social animals that need plenty of space to hop around.",
        metadata={"source": "mammal-pets-doc"},
    ),
]

#### Vector Stores

Vector search is a common way to store and search over unstructured data (such as unstructured text). The idea is to store numeric vectors that are associated with the text. Given a query, we can embed it as a vector of the same dimension and use vector similarity metrics to identify related data in the store.

LangChain VectorStore objects contain methods for adding text and Document objects to the store, and querying them using various similarity metrics. They are often initialized with embedding models, which determine how text data is translated to numeric vectors.

LangChain includes a suite of integrations with different vector store technologies. Some vector stores are hosted by a provider (e.g., various cloud providers) and require specific credentials to use; some (such as Postgres) run in separate infrastructure that can be run locally or via a third-party; others can run in-memory for lightweight workloads. Here we will demonstrate usage of LangChain VectorStores using Chroma, which includes an in-memory implementation.

To instantiate a vector store, we often need to provide an embedding model to specify how text should be converted into a numeric vector. Here we will use OpenAI embeddings.

In [3]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

In [12]:
OpenAIEmbeddings??

Init signature:
OpenAIEmbeddings(
    *,
    client: Any = None,
    async_client: Any = None,
    model: str = 'text-embedding-ada-002',
    dimensions: Optional[int] = None,
    deployment: Optional[str] = 'text-embedding-ada-002',
    api_version: Optional[str] = None,
    base_url: Optional[str] = None,
    openai_api_type: Optional[str] = None,
    openai_proxy: Optional[str] = None,
    embedding_ctx_length: int = 8191,
    api_key: Optional[pydantic.v1.types.SecretStr] = None,
    organization: Optional[str] = None,
    allowed_special: Union[Literal['all'], Set[str], NoneType] = None,
    disallowed_special: Union[Literal['all'], Set[str], Sequence[str], NoneType] = None,
    chunk_size: int = 1000,
    max_retries: int = 2,
    timeout: Union[float, Tuple[float, float], Any, NoneType] = None,
    headers: Any = None,
    tiktoken_enabled: bool = True,
    tiktoken_model_name: Optional[str] = None,
    show_progress_bar: bool = False,
    model_kwargs: Dict[str, Any] = None,
  

In [11]:
vectorstore = Chroma.from_documents(
    documents,
    # embedding=OpenAIEmbeddings(),
    embedding=OpenAIEmbeddings(base_url="http://localhost:1234/v1"),
)

BadRequestError: Error code: 400 - {'error': "'input' field must be a string or an array of strings"}

Calling .from_documents here will add the documents to the vector store. VectorStore implements methods for adding documents that can also be called after the object is instantiated. Most implementations will allow you to connect to an existing vector store-- e.g., by providing a client, index name, or other information. See the documentation for a specific integration for more detail.

Once we've instantiated a VectorStore that contains documents, we can query it. VectorStore includes methods for querying:

- Synchronously and asynchronously
- By string query and by vector
- With and without returning similarity scores
- By similarity and maximum marginal relevance (to balance similarity with query to diversity in retrieved results).

The methods will generally include a list of Document objects in their outputs.

#### Examples

Return documents based on similarity to a string query:

In [ ]:
vectorstore.similarity_search("cat")

Async query:

In [ ]:
await vectorstore.asimilarity_search("cat")

Return scores:

In [ ]:
# Note that providers implement different scores; Chroma here
# returns a distance metric that should vary inversely with
# similarity.

vectorstore.similarity_search_with_score("cat")

Return documents based on similarity to an embedded query:

In [ ]:
embedding = OpenAIEmbeddings().embed_query("cat")

vectorstore.similarity_search_by_vector(embedding)

#### Retrievers

LangChain VectorStore objects do not subclass Runnable, and so cannot immediately be integrated into LangChain Expression Language chains.

LangChain Retrievers are Runnables, so they implement a standard set of methods (e.g., synchronous and asynchronous invoke and batch operations) and are designed to be incorporated in LCEL chains.

We can create a simple version of this ourselves, without subclassing Retriever. If we choose what method we wish to use to retrieve documents, we can create a runnable easily. Below we will build one around the similarity_search method:

In [ ]:
from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda

retriever = RunnableLambda(vectorstore.similarity_search).bind(k=1)  # select top result

retriever.batch(["cat", "shark"])

Vectorstores implement an as_retriever method that will generate a Retriever, specifically a VectorStoreRetriever. These retrievers include specific search_type and search_kwargs attributes that identify what methods of the underlying vector store to call, and how to parameterize them. For instance, we can replicate the above with the following:

In [ ]:
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 1},
)

retriever.batch(["cat", "shark"])

VectorStoreRetriever supports search types of "similarity" (default), "mmr" (maximum marginal relevance, described above), and "similarity_score_threshold". We can use the latter to threshold documents output by the retriever by similarity score.

Retrievers can easily be incorporated into more complex applications, such as retrieval-augmented generation (RAG) applications that combine a given question with retrieved context into a prompt for a LLM. Below we show a minimal example.

In [ ]:
from langchain_openai import ChatOpenAI

# llm = ChatOpenAI(model="gpt-4o-mini")
llm = ChatOpenAI(model="gpt-3.5-turbo")

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

message = """
Answer this question using the provided context only.

{question}

Context:
{context}
"""

prompt = ChatPromptTemplate.from_messages([("human", message)])

rag_chain = {"context": retriever, "question": RunnablePassthrough()} | prompt | llm

In [ ]:
response = rag_chain.invoke("tell me about cats")

print(response.content)